In [1]:
import probtorch
import torch

import combinators
import smc
import ssm
import utils

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
mu = torch.zeros(1, 1)
sigma = torch.ones(1, 1) / 2
delta = torch.ones(1, 1)
zs = torch.ones(1, 50+1) * -1
zs[:, 0] = 0

In [4]:
for t in range(zs.shape[1] - 1):
    zs[:, t+1], mu, sigma, delta, generative = ssm.ssm_step(zs[:, t], mu, sigma, delta, t, generative)

In [5]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [6]:
num_particles = 2500

In [7]:
smc_runner = smc.smc(ssm.ssm_step, ssm.ssm_retrace)

In [8]:
ssm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'sigma': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    }
}

In [9]:
inference, init_ssm = smc.variational_smc(num_particles, ssm.init_ssm, smc_runner, 500, 50, ssm_params, generative)

In [10]:
inference

Trace{'mu': Normal([torch.cuda.FloatTensor of size 2500]), 'sigma': Normal([torch.cuda.FloatTensor of size 2500]), 'delta': Normal([torch.cuda.FloatTensor of size 2500]), 'Z_0': Normal([torch.cuda.FloatTensor of size 2500]), 'Z_1': Normal([torch.cuda.FloatTensor of size 2500]), 'X_1': Normal([torch.cuda.FloatTensor of size 2500x1x1]), 'Z_2': Normal([torch.cuda.FloatTensor of size 2500]), 'X_2': Normal([torch.cuda.FloatTensor of size 2500x1x1]), 'Z_3': Normal([torch.cuda.FloatTensor of size 2500]), 'X_3': Normal([torch.cuda.FloatTensor of size 2500x1x1]), 'Z_4': Normal([torch.cuda.FloatTensor of size 2500]), 'X_4': Normal([torch.cuda.FloatTensor of size 2500x1x1]), 'Z_5': Normal([torch.cuda.FloatTensor of size 2500]), 'X_5': Normal([torch.cuda.FloatTensor of size 2500x1x1]), 'Z_6': Normal([torch.cuda.FloatTensor of size 2500]), 'X_6': Normal([torch.cuda.FloatTensor of size 2500x1x1]), 'Z_7': Normal([torch.cuda.FloatTensor of size 2500]), 'X_7': Normal([torch.cuda.FloatTensor of size 250

In [11]:
generative

Trace{'Z_1': Normal([torch.FloatTensor of size 1x1]), 'X_1': Normal([torch.FloatTensor of size 1x1]), 'Z_2': Normal([torch.FloatTensor of size 1x1]), 'X_2': Normal([torch.FloatTensor of size 1x1]), 'Z_3': Normal([torch.FloatTensor of size 1x1]), 'X_3': Normal([torch.FloatTensor of size 1x1]), 'Z_4': Normal([torch.FloatTensor of size 1x1]), 'X_4': Normal([torch.FloatTensor of size 1x1]), 'Z_5': Normal([torch.FloatTensor of size 1x1]), 'X_5': Normal([torch.FloatTensor of size 1x1]), 'Z_6': Normal([torch.FloatTensor of size 1x1]), 'X_6': Normal([torch.FloatTensor of size 1x1]), 'Z_7': Normal([torch.FloatTensor of size 1x1]), 'X_7': Normal([torch.FloatTensor of size 1x1]), 'Z_8': Normal([torch.FloatTensor of size 1x1]), 'X_8': Normal([torch.FloatTensor of size 1x1]), 'Z_9': Normal([torch.FloatTensor of size 1x1]), 'X_9': Normal([torch.FloatTensor of size 1x1]), 'Z_10': Normal([torch.FloatTensor of size 1x1]), 'X_10': Normal([torch.FloatTensor of size 1x1]), 'Z_11': Normal([torch.FloatTenso

In [12]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

SMC MSE at time 1: 0.089686
SMC MSE at time 2: 0.000171
SMC MSE at time 3: 0.399279
SMC MSE at time 4: 0.487131
SMC MSE at time 5: 0.011356
SMC MSE at time 6: 0.623917
SMC MSE at time 7: 0.136536
SMC MSE at time 8: 0.010016
SMC MSE at time 9: 0.082361
SMC MSE at time 10: 1.525826
SMC MSE at time 11: 2.017944
SMC MSE at time 12: 0.442272
SMC MSE at time 13: 3.585109
SMC MSE at time 14: 0.004128
SMC MSE at time 15: 1.394654
SMC MSE at time 16: 2.339747
SMC MSE at time 17: 5.314420
SMC MSE at time 18: 0.155542
SMC MSE at time 19: 2.414379
SMC MSE at time 20: 1.617982
SMC MSE at time 21: 1.636509
SMC MSE at time 22: 0.634583
SMC MSE at time 23: 4.407403
SMC MSE at time 24: 1.074982
SMC MSE at time 25: 2.038974
SMC MSE at time 26: 0.451838
SMC MSE at time 27: 0.082934
SMC MSE at time 28: 0.703546
SMC MSE at time 29: 2.457116
SMC MSE at time 30: 1.107323
SMC MSE at time 31: 0.695903
SMC MSE at time 32: 0.518482
SMC MSE at time 33: 0.433728
SMC MSE at time 34: 0.311828
SMC MSE at time 35: 0.8